<a href="https://colab.research.google.com/github/Givari17/Weather-Forecasting---Datavidia/blob/main/Ramalan%20Cuaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preprosesing**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mlp
import seaborn as sns
from scipy.stats import chi2_contingency
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

In [ ]:
df_train = pd.read_csv("https://raw.githubusercontent.com/Givari17/Weather-Forecasting---Datavidia/main/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/Givari17/Weather-Forecasting---Datavidia/main/test.csv")
df_train_hourly = pd.read_csv('/content/sample_data/train_hourly.csv', sep=';')
df_test_hourly = pd.read_csv('https://raw.githubusercontent.com/Givari17/Weather-Forecasting---Datavidia/main/test_hourly.csv')


In [ ]:
df_train

In [ ]:
df_train_hourly

## **Memahami Data**

In [ ]:
#memahami data
df_train.info()

In [ ]:
df_train_hourly.info

## **Eliminasi Kolom yang Unik**

In [ ]:
#Eliminasi Kolom UNIQUE
df_train.nunique()

In [ ]:
df_train_hourly.nunique()

## **Mengidentifikasi Data yang Hilang**

In [ ]:
#Identifikasi MISSING DATA
df_train.isna().sum()

In [ ]:
df_train_hourly.isna().sum()

In [ ]:
#Mengisi data yang hilang untuk dataset train
for i in df_train.columns:
    if(df_train[i].dtypes=="object"):
        x = df_train[i].mode()[0]
        df_train[i]=df_train[i].fillna(x)
    else:
        x = df_train[i].mean()
        df_train[i]=df_train[i].fillna(x)

df_train.isna().sum()

In [ ]:

#Mengisi data yang hilang untuk dataset train lyhour
for i in df_train_hourly.columns:
    if(df_train_hourly[i].dtypes=="object"):
        x = df_train_hourly[i].mode()[0]
        df_train_hourly[i]=df_train_hourly[i].fillna(x)
    else:
        x = df_train_hourly[i].mean()
        df_train_hourly[i]=df_train_hourly[i].fillna(x)

df_train_hourly.isna().sum()

## **Menggabungkan Data**

In [ ]:
frames = [df_train, df_train_hourly] 
df = pd.concat(frames).drop_duplicates().reset_index(drop=True)
df

In [ ]:
df.isna().sum()

In [ ]:
#Mengisi data yang hilang untuk dataset GABUNGAN
for i in df.columns:
    if(df[i].dtypes=="object"):
        x = df[i].mode()[0]
        df[i]=df[i].fillna(x)
    else:
        x = df[i].mean()
        df[i]=df[i].fillna(x)

df.isna().sum()

# **Explorasi Data Analysis**

## **Pemisahan Data Kategori dan Kontinu**

In [ ]:
cat = []
con = []

for i in df.columns:
    if(df[i].dtypes=="object"):
        cat.append(i)
    elif(df[i].dtypes=="bool"):
        cat.append(i)
    else:
        con.append(i)

In [ ]:
#Data Kategori
cat

In [ ]:
#Data Kontinu
con

## **Uji Skewness dan Kurtosis**

In [ ]:
import seaborn as sb
mlp.figure(figsize=(20,15))
x = 1

for i in con:
    mlp.subplot(10,5,x)
    sb.distplot(df[i])
    x = x + 1

In [ ]:
df.skew()

In [ ]:
df.var()

## **Uji Korelasi (Bivariate)**

In [ ]:
raw_cor = df.corr()
raw_cor

In [ ]:
raw_cor > 0.0001

In [ ]:
sns.heatmap(df.corr(), cmap="YlGnBu", annot=False);

## **Uji Anova Satu Arah (Analisis Univariate)**

In [ ]:
import matplotlib.pyplot as plt
df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['temperature_2m_max (°C)',
 'temperature_2m_min (°C)',
 'apparent_temperature_max (°C)',
 'apparent_temperature_min (°C)','windgusts_10m_max (km/h)',
 'winddirection_10m_dominant (°)','elevation',])

df_melt.columns = ['index', 'treatments', 'value']


In [ ]:
ax = sns.boxplot(x='treatments', y='value', data=df_melt, color='#99c2a2')
plt.show()

In [ ]:
# stats f_oneway functions mengambil grup sebagai input dan mengembalikan nilai ANOVA F dan p
import scipy.stats as stats
fvalue, pvalue = stats.f_oneway(df['temperature_2m_max (°C)'], 
                                df['temperature_2m_min (°C)'], 
                                df['apparent_temperature_max (°C)'], 
                                df['apparent_temperature_min (°C)'], 
                                df['windgusts_10m_max (km/h)'], 
                                df['winddirection_10m_dominant (°)'],
                                df['elevation'])
print(fvalue)
print(pvalue)



In [ ]:
pvalue < 0.05

## **Feature Selection Menggunakan Chi Square**

In [ ]:
df